In [ ]:
import cv2
import os
import rootutils
from pathlib import Path


rootutils.setup_root(
    os.path.abspath(''), indicator=['.git', 'pyproject.toml'], pythonpath=True
)

from src.data.components.utils import list_files, find_file_by_name

In [ ]:
photo_path = Path('/defect_wrinkles_image_mask/Photos/')
label_path = Path('/defect_wrinkles_image_mask/Outline/')
output_path = Path('/defect_wrinkles_image_mask/Photos_no_background/')

In [3]:
output_path.mkdir(parents=True, exist_ok=True)
image_paths = list_files(photo_path, ['.bmp'])
for image_path in image_paths:
    mask_path = find_file_by_name(label_path, image_path.stem + '_mask', ['.png'])
    image = cv2.imread(str(image_path), cv2.IMREAD_COLOR)
    mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)

    # Ensure the mask is binary
    _, binary_mask = cv2.threshold(mask, 128, 255, cv2.THRESH_BINARY)

    # Create a 3-channel mask for the image
    binary_mask_3c = cv2.merge([binary_mask, binary_mask, binary_mask])

    # Remove the background by applying the mask
    foreground = cv2.bitwise_and(image, binary_mask_3c)

    # Save the result
    output_file = output_path / (image_path.stem + '.bmp')
    cv2.imwrite(str(output_file), foreground)